## Ball Picking Challenge

### Part 1: Navigation

The aim is to program the robot for the first part of the challenge: follow the line from the beginning to the destination point. To do so, you need to reuse the abilities learnt in weeks 1-3; please feel free to reuse the code of those notebooks and exercises.

In [211]:
from packages import initialization
from packages import pioneer3dx as p3dx
import cv2
import numpy

In [212]:
p3dx.init()

Found ROS controller /pioneer3dx_29234_ip_172_31_47_109


In [213]:
## FUNCTIONS ##

#Move robot
def move(V_robot,w_robot):
    r = 0.1953 / 2
    L = 0.33
    w_r = (2 * V_robot + L * w_robot) / (2*r)
    w_l = (2 * V_robot - L * w_robot) / (2*r)
    p3dx.move(w_l, w_r)
    
#Get distances from sonars
def getSonars():
    leftSide = min(p3dx.distance[0:3])
    frontSide = min(p3dx.distance[3:5])
    return leftSide, frontSide

#Get purple or green zones centroid
def get_centroid(image, a):
    hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
    
    #Green has preference
    lower_green = numpy.array([60, 100, 50])
    upper_green = numpy.array([80, 255, 255])
    mask = cv2.inRange(hsv, lower_green, upper_green)
    mask[0:a, 0:150] = 0
    M = cv2.moments(mask)
    area = M['m00']
    
    #If green is not detected, look for purple
    if area == 0 :
        lower_purple = numpy.array([140, 100, 50])
        upper_purple = numpy.array([160, 255, 250])
        mask = cv2.inRange(hsv, lower_purple, upper_purple)
        mask[0:a, 0:150] = 0
        M = cv2.moments(mask)
        area = M['m00']
    
    if area > 0:                   #If area > 0 means that green or purple has been detected
        cx = int(M['m10']/area)
        cy = int(M['m01']/area)
    else:                          #If area == 0 means that no green or purple was detected
        cx = 0
        cy = 0
        
    return area, cx, cy

#Follow the line
def follow_line():
    a = 80 #Initial mask range [0,80]
    print('Following the line')
    while not is_obstacle_detected():
        area, cx, cy = get_centroid(p3dx.image, a) #"a" will determine the mask range [0:a] 
        width = p3dx.image.shape[1]
        linear = 0.2
        Kp = 0.01
        err = cx - (width/2)
        angular = - Kp * err
        if area > 0 :                  #Green or purple has been detected
            move(linear, angular)
            a = 80                     #Use default mask range [0,80]
        else :
            if a == 20 :               #If mask range was augmented before and still nothing was detected
                move(0, 0)             #There is no more to follow -> Stop
            else :                     #If mask wasn't augmented
                move(linear, 0)
                a = 20                 #Change mask range from [0,80] to [0,20]
    print('Obstacle detected')

#Detect obstacles
def is_obstacle_detected():
    threshold = 0.3
    d1 = p3dx.distance[3]
    d2 = p3dx.distance[4]
    if d1<threshold or d2<threshold :
        return True
    else :
        return False

#Get close to the obstacle
def getObstacle():
    threshold = 0.3
    angular = 0.2
    while p3dx.distance[0] > threshold:
        move(0,-angular)
    move(0,0)
    
#Avoid the obstacle
def avoid_obstacle():
    print('Avoiding the obstacle')
    MIN_OBSTACLE_THRESHOLD = 0.2
    MAX_OBSTACLE_THRESHOLD = 0.4
    DEF_X_SPEED = 0.2       #Default forward velocity
    DEF_YAW_SPEED = 0.25    #Default turning velocity
    
    while not is_line_detected():
        leftSide, frontSide = getSonars()
        
        #By default, just move forward
        xSpeed = DEF_X_SPEED
        yawSpeed = DEF_YAW_SPEED
        
        #If we're getting too close to the obstacle with the front side...
        if frontSide <= MIN_OBSTACLE_THRESHOLD:
            #Go backward and turn right quickly (x4)
            xSpeed = -0.1
            yawSpeed = - DEF_YAW_SPEED * 4
        else:
            #If we're getting too close to the obstacle with the left side...
            if leftSide <= MIN_OBSTACLE_THRESHOLD:
                #Move slowly forward (x0.5) and turn right
                xSpeed  = DEF_X_SPEED * 0.5
                yawSpeed = - DEF_YAW_SPEED
            else:
                #If we're getting too far away from the obstacle with the left side...
                if leftSide >= MAX_OBSTACLE_THRESHOLD:
                    #Move slowly forward (x0.5) and turn left
                    xSpeed  = DEF_X_SPEED * 0.5
                    yawSpeed = DEF_YAW_SPEED
        #Execute the movement
        move(xSpeed,yawSpeed)
    print('Line detected')
    
#Detect line to follow
def is_line_detected():
    area, cx, cy = get_centroid(p3dx.image, 80)
    if area == 0 :
        return False
    else :
        return True

#Steer towards the line
def getLine():
    area, cx, cy = get_centroid(p3dx.image, 80)
    while area > 0 :
        move(0.2,0)
        area, cx, cy = get_centroid(p3dx.image, 80)
    p3dx.move(1,-1)
    p3dx.sleep(1)

In [214]:
## MAIN LOOP ##

p3dx.tilt(-0.47)         #Tilt camera
p3dx.gripper(0.05,0.1)   #Open gripper
p3dx.sleep(0.5)          #Wait for gripper
try:
    while True:
        follow_line()
        getObstacle()
        avoid_obstacle()
        getLine()
except KeyboardInterrupt:
    move(0,0)

Following the line
Obstacle detected
Avoiding the obstacle
Line detected
Following the line


---
#### Try-a-Bot: an open source guide for robot programming
Developed by:
[![Robotic Intelligence Lab @ UJI](img/logo/robinlab.png "Robotic Intelligence Lab @ UJI")](http://robinlab.uji.es)

Sponsored by:
<table>
<tr>
<td style="border:1px solid #ffffff ;">
<a href="http://www.ieee-ras.org"><img src="img/logo/ras.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="http://www.cyberbotics.com"><img src="img/logo/cyberbotics.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="http://www.theconstructsim.com"><img src="img/logo/theconstruct.png"></a>
</td>
</tr>
</table>

Follow us:
<table>
<tr>
<td style="border:1px solid #ffffff ;">
<a href="https://www.facebook.com/RobotProgrammingNetwork"><img src="img/logo/facebook.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="https://www.youtube.com/user/robotprogrammingnet"><img src="img/logo/youtube.png"></a>
</td>
</tr>
</table>